# Jupyter Notebook with Java

Run the following code block using a Python kernel. It installs jbang and java kernel.

In [ ]:
!pip3 install jbang # change to pip3 if pip command not found
import jbang
jbang.exec("trust add https://github.com/jupyter-java")
print(jbang.exec("install-kernel@jupyter-java --java 21 --enable-preview").stdout)

# Switch to Java kernel

When the above have completed you should be able to switch to using a Java kernel (some call it runtime) in your Jupyter notebook environment.

You might need to install the Jupyter python package in some environments - particular VSCode based IDE's might not have it directly available.



In [ ]:
!pip install jupyter

When installed try switch kernels. Might have to reload your IDE to have it detect the new available kernels.

# Test it works

Run the following code using a Java kernel

In [1]:
import java.io.File;
import java.io.IOException;

public class RunSpringBootApp {

    private static Process process;

    public static void startApp() {
        String jarPath = "application.jar";
        File logFile = new File("application.log"); // Log file
        ProcessBuilder processBuilder = new ProcessBuilder("java", "-jar", jarPath);
        
        // Redirect of output and errors to a file
        processBuilder.redirectOutput(logFile);
        processBuilder.redirectError(logFile);

        try {
            process = processBuilder.start();
            System.out.println("Spring Boot application started in the background.");
        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void stopApp() {
        if (process != null && process.isAlive()) {
            process.destroy();
            System.out.println("Spring Boot application stopped.");
        } else {
            System.out.println("Application is not running.");
        }
    }

    public static void main(String[] args) {
        startApp();
        // To stop the app run stopApp() in another cell
    }
}


In [2]:
RunSpringBootApp.startApp();

Spring Boot application started in the background.


In [9]:
RunSpringBootApp.stopApp();


Spring Boot application stopped.


In [4]:
import java.io.BufferedReader;
import java.io.File;
import java.io.FileReader;
import java.io.IOException;
import java.util.LinkedList;

public class LogReader {

    public static void printLastNLogs(int n) {
        File logFile = new File("application.log");
        LinkedList<String> lastNLines = new LinkedList<>();

        try (BufferedReader reader = new BufferedReader(new FileReader(logFile))) {
            String line;
            while ((line = reader.readLine()) != null) {
                lastNLines.add(line);
                if (lastNLines.size() > n) {
                    lastNLines.removeFirst();  // Maintain only the last N lines
                }
            }
        } catch (IOException e) {
            e.printStackTrace();
        }

        // Print the last N lines
        for (String logLine : lastNLines) {
            System.out.println(logLine);
        }
    }
}


In [5]:
LogReader.printLastNLogs(35);


  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/
 :: Spring Boot ::                (v3.1.5)

2024-08-29T18:10:58.855+03:00  INFO 14547 --- [           main] c.e.a.AccountingDemoApplication          : Starting AccountingDemoApplication v3.0.x-SNAPSHOT using Java 17.0.10 with PID 14547 (/home/slava/projects/demo-jupyternb/res/application.jar started by slava in /home/slava/projects/demo-jupyternb/res)
2024-08-29T18:10:58.858+03:00  INFO 14547 --- [           main] c.e.a.AccountingDemoApplication          : No active profile set, falling back to 1 default profile: "default"
2024-08-29T18:10:59.999+03:00  INFO 14547 --- [           main] o.s.b.w.embedded.tomcat.TomcatWebServer  : Tomcat initialized with port(s): 8080 (http)
2024-08-29T18:11:00.009+03:00  INFO 14547 --- [           main] o.apa

In [6]:
%classpath /home/slava/projects/demo-jupyternb/res/

import com.example.accounting_demo.service.EntityService;


In [7]:
import java.lang.reflect.Method;
import java.rmi.RemoteException;
import java.rmi.registry.LocateRegistry;
import java.rmi.registry.Registry;
import java.lang.reflect.InvocationTargetException;


public class EntityServiceClient {

    private static EntityService getEntityService() throws Exception {
        try {
            Registry registry = LocateRegistry.getRegistry("localhost", 1099);
            return (EntityService) registry.lookup("EntityService");
        } catch (RemoteException e) {
            throw new Exception("Failed to connect to RMI registry or lookup service", e);
        }
    }

    public static Object invokeMethod(String methodName, Object... args) throws Exception {
        try {
            EntityService service = getEntityService();
            Class<?>[] argTypes = new Class[args.length];
            for (int i = 0; i < args.length; i++) {
                argTypes[i] = args[i].getClass();
            }
            Method method = service.getClass().getMethod(methodName, argTypes);
            return method.invoke(service, args);
        } catch (NoSuchMethodException e) {
            throw new Exception("No such method found: " + methodName, e);
        } catch (IllegalAccessException e) {
            throw new Exception("Illegal access to method: " + methodName, e);
        } catch (InvocationTargetException e) {
            throw new Exception("Method invocation failed: " + methodName, e);
        } catch (RemoteException e) {
            throw new Exception("Remote method invocation failed", e);
        }
    }
}



In [8]:
String result = (String) EntityServiceClient.invokeMethod("demoMethod");
System.out.println("Result from EntityService: " + result);


Result from EntityService: Hello from EntityService!
